In [3]:
import torch
import torch.nn as nn
from torch.nn import functional as f
from pathlib import Path
import numpy as np
import torchvision
from torch.utils.data import DataLoader, Dataset
import os
from torchsummary import torchsummary

In [12]:
ORIGINAL_IMAGE = '/import/mqhome/duanct/dataSets/NDI_images/20220725/20220725/Observed_Crop_200x200pix'
TARGET_IMAGE = '/import/mqhome/duanct/dataSets/NDI_images/20220725/20220725/Calculated_200x200/grayscale'
POSITIVE_RATIO = 0.1
BATCH_SIZE = 8

## CosineEmbeddingLoss

In [4]:
class NDI_dataset(Dataset):
    def __init__(self, positive_ratio=0.5, times=10):
        original_images = list(Path(ORIGINAL_IMAGE).glob('*.jpg'))
        target_images = list(Path(TARGET_IMAGE).glob('*.jpg'))
        self.origins, self.targets, self.label = [], [], []
        for original_image in original_images:
            self.origins.append(torchvision.io.read_image(str(original_image)))
            self.targets.append(torchvision.io.read_image(str(Path.joinpath(Path(TARGET_IMAGE), original_image.name.split('_')[0] + '.jpg'))))
            self.label.append(torch.tensor(1))
        for _ in range(times):
            for original_image in original_images:
                self.origins.append(torchvision.io.read_image(str(original_image)))
                self.targets.append(torchvision.io.read_image(str(np.random.choice(target_images))))
                self.label.append(torch.tensor(-1))

    def __getitem__(self, idx):
        return self.origins[idx], self.targets[idx], self.label[idx]

    def __len__(self):
        return len(self.origins)

In [5]:
train_iter = DataLoader(NDI_dataset(POSITIVE_RATIO), BATCH_SIZE, shuffle=True)

In [6]:
model = torchvision.models.resnet18(pretrained=False)

In [7]:
model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
model.fc = nn.Linear(512, 256)

In [8]:
torchsummary.summary(model, input_size=(1, 200, 200), device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 100, 100]           3,136
       BatchNorm2d-2         [-1, 64, 100, 100]             128
              ReLU-3         [-1, 64, 100, 100]               0
         MaxPool2d-4           [-1, 64, 50, 50]               0
            Conv2d-5           [-1, 64, 50, 50]          36,864
       BatchNorm2d-6           [-1, 64, 50, 50]             128
              ReLU-7           [-1, 64, 50, 50]               0
            Conv2d-8           [-1, 64, 50, 50]          36,864
       BatchNorm2d-9           [-1, 64, 50, 50]             128
             ReLU-10           [-1, 64, 50, 50]               0
       BasicBlock-11           [-1, 64, 50, 50]               0
           Conv2d-12           [-1, 64, 50, 50]          36,864
      BatchNorm2d-13           [-1, 64, 50, 50]             128
             ReLU-14           [-1, 64,

In [9]:
device = torch.device('cuda')
lr = 0.001
EPOCHS = 20

loss = nn.CosineEmbeddingLoss(margin=0.3)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
model.to(device)
train_iter = DataLoader(NDI_dataset(POSITIVE_RATIO), BATCH_SIZE, shuffle=True)
for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    for origin, target, label in train_iter:
        optimizer.zero_grad()
        origin, target = origin.float(), target.float()
        origin, target, label = origin.to(device), target.to(device), label.to(device)
        origin_feature = model(origin)
        target_feature = model(target)
        l = loss(origin_feature, target_feature, label)
        l.backward()
        total_loss += l
    print(f'Epoch: {epoch + 1}, Ave. Loss: {total_loss / len(train_iter):.4f}')


Epoch: 1, Ave. Loss: 0.5940
Epoch: 2, Ave. Loss: 0.5952
Epoch: 3, Ave. Loss: 0.5944
Epoch: 4, Ave. Loss: 0.5929
Epoch: 5, Ave. Loss: 0.5938
Epoch: 6, Ave. Loss: 0.5953
Epoch: 7, Ave. Loss: 0.5931
Epoch: 8, Ave. Loss: 0.5945
Epoch: 9, Ave. Loss: 0.5908
Epoch: 10, Ave. Loss: 0.5949
Epoch: 11, Ave. Loss: 0.5910
Epoch: 12, Ave. Loss: 0.5936
Epoch: 13, Ave. Loss: 0.5947
Epoch: 14, Ave. Loss: 0.5938
Epoch: 15, Ave. Loss: 0.5946
Epoch: 16, Ave. Loss: 0.5942
Epoch: 17, Ave. Loss: 0.5944
Epoch: 18, Ave. Loss: 0.5940
Epoch: 19, Ave. Loss: 0.5937
Epoch: 20, Ave. Loss: 0.5939


In [10]:
from PIL import Image

model.to('cpu')
original_images = list(Path(ORIGINAL_IMAGE).glob('*.jpg'))
original_image_name = np.random.choice(original_images)
original_image = torchvision.io.read_image(str(original_image_name)).unsqueeze_(0).float()
target_image = torchvision.io.read_image(str(Path.joinpath(Path(TARGET_IMAGE), original_image_name.name.split('_')[0] + '.jpg'))).unsqueeze_(0).float()
print(nn.CosineSimilarity()(model(original_image), model(target_image)))

tensor([0.9949], grad_fn=<DivBackward0>)


In [11]:
model.to('cpu')
original_images = list(Path(ORIGINAL_IMAGE).glob('*.jpg'))
target_images = list(Path(TARGET_IMAGE).glob('*.jpg'))
original_image_name = np.random.choice(original_images)
target_image_name = np.random.choice(target_images)
original_image = torchvision.io.read_image(str(original_image_name)).unsqueeze_(0).float()
target_image = torchvision.io.read_image(str(target_image_name)).unsqueeze_(0).float()
print(nn.CosineSimilarity()(model(original_image), model(target_image)))

tensor([0.9946], grad_fn=<DivBackward0>)


In [129]:
dimensions = 512
a = torch.rand((1, dimensions))
b = torch.rand((1, dimensions))
nn.CosineSimilarity()(a, b)

tensor([0.7585])

## Binary Classification

In [7]:
a = torch.rand((8, 512))
b = torch.rand((8, 512))
torch.cat([a, b], dim=).size()

torch.Size([16, 512])

In [11]:
class ClassificationModel(nn.Module):
    def __init__(self):
        super(ClassificationModel, self).__init__()
        self.backbone = torchvision.models.resnet18(pretrained=False)
        self.backbone.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.backbone.fc = nn.Linear(512, 256)
        self.classifier = nn.Linear(512, 2)
    def forward(self, origin, target):
        origin_feature = f.relu(self.backbone(origin))
        target_feature = f.relu(self.backbone(target))
        return self.classifier(torch.cat([origin_feature, target_feature], dim=-1))

In [12]:
class NDI_dataset_classifier(Dataset):
    def __init__(self, positive_ratio=0.5, times=10):
        original_images = list(Path(ORIGINAL_IMAGE).glob('*.jpg'))
        target_images = list(Path(TARGET_IMAGE).glob('*.jpg'))
        self.origins, self.targets, self.label = [], [], []
        for original_image in original_images:
            self.origins.append(torchvision.io.read_image(str(original_image)))
            self.targets.append(torchvision.io.read_image(str(Path.joinpath(Path(TARGET_IMAGE), original_image.name.split('_')[0] + '.jpg'))))
            self.label.append(torch.tensor(1, dtype=torch.long))
        for _ in range(times):
            for original_image in original_images:
                self.origins.append(torchvision.io.read_image(str(original_image)))
                self.targets.append(torchvision.io.read_image(str(np.random.choice(target_images))))
                self.label.append(torch.tensor(0, dtype=torch.long))

    def __getitem__(self, idx):
        return self.origins[idx], self.targets[idx], self.label[idx]

    def __len__(self):
        return len(self.origins)

In [16]:
model = ClassificationModel()
torchsummary.summary(model, input_size=[(1, 200, 200), (1, 200, 200)], device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 100, 100]           3,136
       BatchNorm2d-2         [-1, 64, 100, 100]             128
              ReLU-3         [-1, 64, 100, 100]               0
         MaxPool2d-4           [-1, 64, 50, 50]               0
            Conv2d-5           [-1, 64, 50, 50]          36,864
       BatchNorm2d-6           [-1, 64, 50, 50]             128
              ReLU-7           [-1, 64, 50, 50]               0
            Conv2d-8           [-1, 64, 50, 50]          36,864
       BatchNorm2d-9           [-1, 64, 50, 50]             128
             ReLU-10           [-1, 64, 50, 50]               0
       BasicBlock-11           [-1, 64, 50, 50]               0
           Conv2d-12           [-1, 64, 50, 50]          36,864
      BatchNorm2d-13           [-1, 64, 50, 50]             128
             ReLU-14           [-1, 64,

In [ ]:
device = torch.device('cuda')
lr = 0.001
EPOCHS = 20

loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
model.to(device)
train_iter = DataLoader(NDI_dataset(POSITIVE_RATIO), BATCH_SIZE, shuffle=True)
for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    for origin, target, label in train_iter:
        optimizer.zero_grad()
        origin, target = origin.float(), target.float()
        origin, target, label = origin.to(device), target.to(device), label.to(device)
        origin_feature = model(origin)
        target_feature = model(target)
        l = loss(origin_feature, target_feature, label)
        l.backward()
        total_loss += l
    print(f'Epoch: {epoch + 1}, Ave. Loss: {total_loss / len(train_iter):.4f}')

In [17]:
x = torch.rand((1, 2))

In [20]:
torch.argmax(x).type(x.dtype).sum()

tensor(0.)

In [23]:
torch.rand(1)

tensor([0.6341])

In [8]:
mnist_dataset = torchvision.datasets.FashionMNIST(root='../data', train=True, transform=torchvision.transforms.Compose([torchvision.transforms.ToTensor()]), download=True)
mnist_loader = torch.utils.data.DataLoader(mnist_dataset, batch_size=32, shuffle=True)
for x, y in mnist_loader:
    print(x.shape)
    print(y.shape)
    break

torch.Size([32, 1, 28, 28])
torch.Size([32])


In [11]:
x[0].shape

torch.Size([1, 28, 28])

In [ ]:
class NDIDatasetContrastiveLearning(Dataset):
    def __init__(self):
        super(NDIDatasetContrastiveLearning, self).__init__()
        original_images = list(Path(ORIGINAL_IMAGE).glob('*.jpg'))
        target_images = list(Path(TARGET_IMAGE).glob('*.jpg'))
        self.origins, self.targets, self.label = [], [], []

In [13]:
original_images = list(Path(ORIGINAL_IMAGE).glob('*.jpg'))
img_name = np.random.choice(original_images)
img = torchvision.io.read_image(img_name)
img.shape

ValueError: 'a' cannot be empty unless no samples are taken

In [14]:
original_images

[]